# Model Building

1. Train-test split

2. Multiple Linear Regression

3. Lasso Regression

4. Random Forest Regression

5. Gradient Boosting Regression

6. Bagging Regression

7. Tune models using GridsearchCV

8. Test Ensembles
